In [1]:
# Montando o drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [2]:
### importando bibliotecas que iremos utilizar
import pandas as pd
import datetime as dt
import numpy as np

In [3]:
# importando arquivos de trabalho
predictions = pd.DataFrame(pd.read_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/bid tuner utils/myfilename.csv'))
base_bids = pd.DataFrame(pd.read_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/bid tuner utils/base_bids.csv'))

In [4]:
base_bids

,partner_reference,BR,US
0,OMN-4273,3,14
1,OMN-3185,15,46
2,JNP-JP408570,16,20
3,OMN-4480,5,48
4,JNP-JP335495,54,49
...,...,...,...
108677,JNP-JP077769,67,48
108678,JNP-JP077766,67,49
108679,JNP-JP077761,0,0
108680,JNP-JP077757,67,49


In [5]:
predictions

,tps_predictions,pos,sku,report_date,base_bid_eur,tps_predictions_smooth
0,0.109835,BR,OMN-1009,2021-09-28 00:00:00+00:00,0.78,0.128345
1,0.102889,BR,OMN-1009,2021-09-28 00:00:00+00:00,0.77,0.120309
2,0.095811,BR,OMN-1009,2021-09-28 00:00:00+00:00,0.76,0.112105
3,0.399359,BR,OMN-10179,2021-09-28 00:00:00+00:00,0.73,0.437591
4,0.384722,BR,OMN-10179,2021-09-28 00:00:00+00:00,0.71,0.423582
...,...,...,...,...,...,...
848,-0.040309,BR,OMN-989,2021-09-28 00:00:00+00:00,0.48,0.000000
849,-0.045435,BR,OMN-989,2021-09-28 00:00:00+00:00,0.47,0.000000
850,0.290989,BR,OMN-9899,2021-09-28 00:00:00+00:00,0.45,0.329184
851,0.312587,BR,OMN-9899,2021-09-28 00:00:00+00:00,0.46,0.351619


In [6]:
# Criando métrica ratio = TPS/valor do bid
predictions['smooth_ratio'] = predictions['tps_predictions_smooth'] / predictions['base_bid_eur']
predictions.sort_values(by = 'smooth_ratio', ascending= False)

,tps_predictions,pos,sku,report_date,base_bid_eur,tps_predictions_smooth,smooth_ratio
827,0.667505,BR,OMN-9475,2021-09-28 00:00:00+00:00,0.22,0.655179,2.978084
826,0.702915,BR,OMN-9475,2021-09-28 00:00:00+00:00,0.23,0.678282,2.949052
828,0.743912,BR,OMN-9475,2021-09-28 00:00:00+00:00,0.24,0.703449,2.931039
819,0.251599,BR,OMN-9341,2021-09-28 00:00:00+00:00,0.17,0.287307,1.690041
818,0.266868,BR,OMN-9341,2021-09-28 00:00:00+00:00,0.18,0.303681,1.687117
...,...,...,...,...,...,...,...
583,-0.000475,BR,OMN-6227,2021-09-28 00:00:00+00:00,0.25,0.000000,0.000000
164,-0.027601,BR,OMN-2043,2021-09-28 00:00:00+00:00,0.37,0.000000,0.000000
291,-0.001477,BR,OMN-3007,2021-09-28 00:00:00+00:00,0.55,0.000000,0.000000
292,-0.005905,BR,OMN-3007,2021-09-28 00:00:00+00:00,0.54,0.000000,0.000000


In [7]:
# Filtrando tabela para que fiquemos apenas com os bids que oferecem o maior ratio
predictions = predictions.sort_values('smooth_ratio', ascending = False).drop_duplicates('sku')
predictions

,tps_predictions,pos,sku,report_date,base_bid_eur,tps_predictions_smooth,smooth_ratio
827,0.667505,BR,OMN-9475,2021-09-28 00:00:00+00:00,0.22,0.655179,2.978084
819,0.251599,BR,OMN-9341,2021-09-28 00:00:00+00:00,0.17,0.287307,1.690041
333,0.372877,BR,OMN-3405,2021-09-28 00:00:00+00:00,0.25,0.412093,1.648374
14,0.413425,BR,OMN-10533,2021-09-28 00:00:00+00:00,0.28,0.450857,1.610202
241,0.791301,BR,OMN-2445,2021-09-28 00:00:00+00:00,0.53,0.730491,1.378284
...,...,...,...,...,...,...,...
416,-0.047346,BR,OMN-4663,2021-09-28 00:00:00+00:00,0.22,0.000000,0.000000
438,-0.148166,BR,OMN-4955,2021-09-28 00:00:00+00:00,0.01,0.000000,0.000000
798,-0.012419,BR,OMN-9071,2021-09-28 00:00:00+00:00,0.56,0.000000,0.000000
248,-0.026682,BR,OMN-2452,2021-09-28 00:00:00+00:00,0.28,0.000000,0.000000


In [8]:
# Pegando apenas pedaço da tabela de bids que está nas previsões
bids_subset = base_bids[base_bids['partner_reference'].isin(predictions['sku'])]
bids_subset

,partner_reference,BR,US
10,OMN-6025,57,23
13,OMN-8598,39,42
16,OMN-4609,72,42
27,OMN-2087,21,33
34,OMN-2081,49,28
...,...,...,...
74799,OMN-5591,73,50
74885,OMN-5465,17,50
74953,OMN-5718,47,48
74976,OMN-5739,44,50


In [9]:
bids_subset_merged = bids_subset.merge(predictions,left_on= 'partner_reference', right_on= 'sku', how = 'left')
bids_subset_merged

,partner_reference,BR,US,tps_predictions,pos,sku,report_date,base_bid_eur,tps_predictions_smooth,smooth_ratio
0,OMN-6025,57,23,0.279494,BR,OMN-6025,2021-09-28 00:00:00+00:00,0.58,0.317088,0.546703
1,OMN-8598,39,42,0.229145,BR,OMN-8598,2021-09-28 00:00:00+00:00,0.38,0.262922,0.691901
2,OMN-4609,72,42,0.528382,BR,OMN-4609,2021-09-28 00:00:00+00:00,0.71,0.551719,0.777069
3,OMN-2087,21,33,0.089563,BR,OMN-2087,2021-09-28 00:00:00+00:00,0.22,0.104850,0.476589
4,OMN-2081,49,28,0.334603,BR,OMN-2081,2021-09-28 00:00:00+00:00,0.50,0.374080,0.748160
...,...,...,...,...,...,...,...,...,...,...
280,OMN-5591,73,50,-0.026675,BR,OMN-5591,2021-09-28 00:00:00+00:00,0.72,0.000000,0.000000
281,OMN-5465,17,50,0.033528,BR,OMN-5465,2021-09-28 00:00:00+00:00,0.18,0.039375,0.218749
282,OMN-5718,47,48,-0.013985,BR,OMN-5718,2021-09-28 00:00:00+00:00,0.46,0.000000,0.000000
283,OMN-5739,44,50,0.017354,BR,OMN-5739,2021-09-28 00:00:00+00:00,0.45,0.020389,0.045308


In [10]:
bids_subset_merged = bids_subset_merged[['partner_reference', 'base_bid_eur', 'US']]
bids_subset_merged

,partner_reference,base_bid_eur,US
0,OMN-6025,0.58,23
1,OMN-8598,0.38,42
2,OMN-4609,0.71,42
3,OMN-2087,0.22,33
4,OMN-2081,0.50,28
...,...,...,...
280,OMN-5591,0.72,50
281,OMN-5465,0.18,50
282,OMN-5718,0.46,48
283,OMN-5739,0.45,50


In [11]:
bids_subset_merged = bids_subset_merged.rename(columns = {'base_bid_eur': 'BR'})
bids_subset_merged


,partner_reference,BR,US
0,OMN-6025,0.58,23
1,OMN-8598,0.38,42
2,OMN-4609,0.71,42
3,OMN-2087,0.22,33
4,OMN-2081,0.50,28
...,...,...,...
280,OMN-5591,0.72,50
281,OMN-5465,0.18,50
282,OMN-5718,0.46,48
283,OMN-5739,0.45,50


In [12]:
bids_subset_merged['BR'] = bids_subset_merged['BR'] * 100 
bids_subset_merged['BR'] = bids_subset_merged['BR'].astype(int)
bids_subset_merged

,partner_reference,BR,US
0,OMN-6025,57,23
1,OMN-8598,38,42
2,OMN-4609,71,42
3,OMN-2087,22,33
4,OMN-2081,50,28
...,...,...,...
280,OMN-5591,72,50
281,OMN-5465,18,50
282,OMN-5718,46,48
283,OMN-5739,45,50


In [13]:
base_bids

,partner_reference,BR,US
0,OMN-4273,3,14
1,OMN-3185,15,46
2,JNP-JP408570,16,20
3,OMN-4480,5,48
4,JNP-JP335495,54,49
...,...,...,...
108677,JNP-JP077769,67,48
108678,JNP-JP077766,67,49
108679,JNP-JP077761,0,0
108680,JNP-JP077757,67,49


In [14]:
base_bids_not_in_prediction = base_bids.loc[~base_bids['partner_reference'].isin(bids_subset_merged['partner_reference'])]
base_bids_not_in_prediction

,partner_reference,BR,US
0,OMN-4273,3,14
1,OMN-3185,15,46
2,JNP-JP408570,16,20
3,OMN-4480,5,48
4,JNP-JP335495,54,49
...,...,...,...
108677,JNP-JP077769,67,48
108678,JNP-JP077766,67,49
108679,JNP-JP077761,0,0
108680,JNP-JP077757,67,49


In [15]:
bids_output = pd.concat([base_bids_not_in_prediction, bids_subset_merged]).reset_index(drop = True)
bids_output

,partner_reference,BR,US
0,OMN-4273,3,14
1,OMN-3185,15,46
2,JNP-JP408570,16,20
3,OMN-4480,5,48
4,JNP-JP335495,54,49
...,...,...,...
108677,OMN-5591,72,50
108678,OMN-5465,18,50
108679,OMN-5718,46,48
108680,OMN-5739,45,50


In [16]:
# CUSPINDO PLANILHA FINAL
dia = (dt.datetime.today() + dt.timedelta(days=1)).strftime(format='%d_%m_%y')

# Cospe tabela de bids
bids_output.to_csv('/gdrive/My Drive/Planilha de Bids trivago/Fase 2/bid tuner utils/bid_final_mvp_' + dia + '.csv', sep=',', decimal='.', 
                              encoding="utf-8-sig", index = False)

In [17]:
bids_output

,partner_reference,BR,US
0,OMN-4273,3,14
1,OMN-3185,15,46
2,JNP-JP408570,16,20
3,OMN-4480,5,48
4,JNP-JP335495,54,49
...,...,...,...
108677,OMN-5591,72,50
108678,OMN-5465,18,50
108679,OMN-5718,46,48
108680,OMN-5739,45,50
